In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers

from pathlib import Path
# from tqdm.notebook import tqdm
from imageio import imread

import seaborn as sns


In [ ]:
def read_grayscale_pngs(path, width=20, height=13):
    path = Path(path)
    if not path.exists():
        print("Path {} doesn't exist".format(path))
        return None

    # print(len([name for name in os.listdir('{}/.'.format(path)) if os.path.isfile(name)]))
    num_files = len(list(path.glob('**/*.png'))) # Calculate amount of files in directory
    # num_files = len([f for f in path.iterdir() if path.joinpath(f).is_file()]) # Calculate amount of files in directory

    images = np.empty((num_files, 13, 20))

    for i, image_path in enumerate(sorted(path.glob('**/*.png'), key=lambda f: int(f.stem))):
        images[i] = np.array(imread(image_path))[:, :, 0] # Pixel data: It's grayscale so take only Red values from [R, G, B, A]
    return images

In [8]:
legal = read_grayscale_pngs("out/legal/orig")
illegal = read_grayscale_pngs("out/illegal/orig")  

mirrored_legal = read_grayscale_pngs("out/legal/mirrored")
mirrored_illegal = read_grayscale_pngs("out/illegal/mirrored")

rotated20legal = read_grayscale_pngs("out/legal/rotated20.0")
rotated20illegal = read_grayscale_pngs("out/illegal/rotated20.0")

rotatedm20legal = read_grayscale_pngs("out/legal/rotated-20.0")
rotatedm20illegal = read_grayscale_pngs("out/illegal/rotated-20.0")

legal_test = read_grayscale_pngs("testing/legal")
illegal_test = read_grayscale_pngs("testing/illegal")


In [15]:
from sklearn.model_selection import train_test_split# Spliiting data into test and train sets

x_train_legal = np.concatenate((legal, mirrored_legal, rotated20legal, rotatedm20legal))
x_train_illegal = np.concatenate((illegal, mirrored_illegal, rotated20illegal, rotatedm20illegal))
X_train = np.concatenate((x_train_legal, x_train_illegal))    

X_train = X_train / 255.0
Y_train = np.concatenate((np.full(len(x_train_legal),0), np.full(len(x_train_illegal), 1)))


X_test = np.concatenate((legal_test, illegal_test))
X_test = X_test / 255.0
Y_test = np.concatenate((np.full(len(legal_test), 0), np.full(len(illegal_test), 1)))

# X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.10, random_state=42)# fitting the model

In [26]:
# Convolutional
keras.backend.clear_session()
modelr = keras.Sequential()

modelr.add(layers.Reshape((1,260), input_shape=(13,20)))
# modelr.add(layers.Flatten())
modelr.add(layers.LSTM(160))
modelr.add(layers.Dense(100, activation="relu"))
modelr.add(layers.Dense(1,  activation="sigmoid"))

modelr.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
modelr.fit(X_train, Y_train, shuffle=False, epochs=10, batch_size=200, validation_split=0.1)

loss, accuracy = modelr.evaluate(X_test, Y_test, verbose=0)
    
print(loss, accuracy*100)

Epoch 1/10
63/63 [==============================] - 2s 12ms/step - loss: 1.8508 - accuracy: 0.5963 - val_loss: 0.6902 - val_accuracy: 0.6346
Epoch 2/10
63/63 [==============================] - 0s 6ms/step - loss: 0.7236 - accuracy: 0.3733 - val_loss: 0.6948 - val_accuracy: 0.3855
Epoch 3/10
63/63 [==============================] - 0s 6ms/step - loss: 0.6467 - accuracy: 0.7872 - val_loss: 0.3803 - val_accuracy: 1.0000
Epoch 4/10
63/63 [==============================] - 0s 6ms/step - loss: 0.6500 - accuracy: 0.5359 - val_loss: 0.2330 - val_accuracy: 0.9864
Epoch 5/10
63/63 [==============================] - 0s 6ms/step - loss: 0.4680 - accuracy: 0.8843 - val_loss: 0.0213 - val_accuracy: 0.9957
Epoch 6/10
63/63 [==============================] - 0s 6ms/step - loss: 0.3890 - accuracy: 0.8366 - val_loss: 0.0420 - val_accuracy: 0.9842
Epoch 7/10
63/63 [==============================] - 0s 6ms/step - loss: 0.2451 - accuracy: 0.9378 - val_loss: 0.0637 - val_accuracy: 0.9770
Epoch 8/10
63/63 [=